In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import numpy as np
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit  
from pyspark.sql.types import StructType, StructField, StringType

class PrcCargaInterfaces():
  
  def __init__(self, id_solicitud):
    self.id_solicitud = id_solicitud
    
    #Obteniendo parametros
    parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
    self.fecha_proceso = parametros['fecha_proceso']  
    self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
    self.archivo_entrada = parametros['archivo_entrada']
    self.tabla_destino = parametros['tabla_destino']
    self.tabla_destino_2 = parametros['tabla_destino_2']
    self.ruta_origen = '/dbfs/mnt/ereportinterfaz/inseir/'
        
    #Fecha de actualizacion
    fecha_actualizacion = datetime.now()
    self.fecha_actualizacion = fecha_actualizacion
    self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
    
    #Conexion
    connection = PrcGeneral().databaseConnect()
    self.connection = connection
    
    cursor = self.connection.cursor()
    self.cursor = cursor
    
      
  def cargarInterfaceFormatoRcd(self):    
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [12,9,12,80,15,4,15,11,1,1,12,2,2,2,2,18,14,1,11,2,2,2,2,2,8,1,1,1,2,10,2,12]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['NUM_DOC','CODIGO_SBS','NUM_TARJETA','NOMBRE','SIGLAS','CODIGO_CIUU','REG_PUBLICO','RUC','PFT','TIPO_DOCUMENTO','NUM_DOCUMENTO','RESIDENCIA','CALIF_CREDITICIA','DFT','MONEDA','SALDO','MONTO_CREDITO','TIPO_DOC_TRIBUT','NUM_DOC_TRIBUT','FILLER','TIP_OPE','MAGNITUD_EMP','ACCIONISTA','TIP_TRIB','NUM_TRIB','SEXO','EST_CIVIL','TIPO_CLIENTE','PAIS','FECHA_NACIMIENTO','TIPO_DOC_COMPLEMENTARIO','NUM_DOC_COMPLEMENTARIO'], 
                          dtype = {'NUM_DOC': str,
                                   'CODIGO_SBS': str,
                                   'NUM_TARJETA': str,
                                   'NOMBRE': str,
                                   'SIGLAS': str,
                                   'CODIGO_CIUU': str,
                                   'REG_PUBLICO': str,
                                   'RUC': str,
                                   'PFT': str,
                                   'TIPO_DOCUMENTO': str,
                                   'NUM_DOCUMENTO': str,
                                   'RESIDENCIA': str,
                                   'CALIF_CREDITICIA': str,
                                   'DFT': str,
                                   'MONEDA': str,
                                   'SALDO': str,
                                   'MONTO_CREDITO': str,
                                   'TIPO_DOC_TRIBUT': str,
                                   'NUM_DOC_TRIBUT': str,
                                   'FILLER': str,
                                   'TIP_OPE': str,
                                   'MAGNITUD_EMP': str,
                                   'ACCIONISTA': str,
                                   'TIP_TRIB': str,
                                   'NUM_TRIB': str,
                                   'SEXO': str,
                                   'EST_CIVIL': str,
                                   'TIPO_CLIENTE': str,
                                   'PAIS': str,
                                   'FECHA_NACIMIENTO': str,
                                   'TIPO_DOC_COMPLEMENTARIO': str,
                                   'NUM_DOC_COMPLEMENTARIO': str}) 
   
    #Insertando datos
    for df in df_full:  
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str    
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    
    self.cursor.close()   
    

  def cargarInterfaceSatMaeFusion(self):    
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [12,4,1,1,1,1,1,4,40,40,40,8,6,40,40,2,12,1,10,13]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['NRO_DOC','CIIU','RESIDENCIA','MAGNITUD','TIPO_PERSONA','SEXO','ESTADO','PAIS','AP_PATERNO','AP_MATERNO','NOMBRE','FE_NACI','FE_UINF','NOMBRE2','APE_CASA','TIPO_DOC_COMP','NRO_DOC_COMP','TIPO_CLIENTE','CODIGO_SBS','RUT'],
                          dtype = {'NRO_DOC': str,
                                   'CIIU': str,
                                   'RESIDENCIA': str,
                                   'MAGNITUD': str,
                                   'TIPO_PERSONA': str,
                                   'SEXO': str,
                                   'ESTADO': str,
                                   'PAIS': str,
                                   'AP_PATERNO': str,
                                   'AP_MATERNO': str,
                                   'NOMBRE': str,
                                   'FE_NACI': str,
                                   'FE_UINF': str,
                                   'NOMBRE2': str,
                                   'APE_CASA': str,
                                   'TIPO_DOC_COMP': str,
                                   'NRO_DOC_COMP': str,
                                   'TIPO_CLIENTE': str,
                                   'CODIGO_SBS': str,
                                   'RUT': str})
    
    #Insertando datos
    for df in df_full:  
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, df.values.tolist())
        self.connection.commit()    
    self.cursor.close()  
    
    
  def cargarInterfaceSatIndiceMorosidad(self):
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [8,12,4,9,4]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['FECHA','CUENTA_CMR','DIAS_MORA','CAPITAL','TIPO_PRODUCTO'],
                          dtype = {'FECHA': str,
                                    'CUENTA_CMR': str,
                                    'DIAS_MORA': str,
                                    'CAPITAL': float,
                                    'TIPO_PRODUCTO': str})
    
    #Insertando datos
    for df in df_full:     
      #df['CUENTA_CMR']=df['CUENTA_CMR'].astype(str)
      df['FECHA']=pd.to_datetime(df['FECHA'], format='%Y%m%d').dt.date
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?)"
      values = df.values.tolist()
      if values:
        self.cursor.fast_executemany = True
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()  
      
    
  def cargarInterfaceMancomunoIssac(self):   
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [20,2,11,1,12,1,1,1,1,1,4,1,1,20,120,40,40,40]
    df_full = pd.read_fwf(file_name, 
                     widths = fwidths,
                     chunksize = 15000,
                     names = ['NUM_CMR','TIPO_TRIBUTARIO','DOC_TRIBUTARIO','TIPO_DOC','NUM_DOC','TIPO_PERSONA','RESIDENCIA','MAGNITUD','ACCIONISTA','RELACION','PAIS','GENERO','EDOCIVIL','ESPACIO','APE_PAT','APE_MAT','PRIMER_NOMBRE','SEGUNDO_NOMBRE'],
                    dtype = {'NUM_CMR': str,
                             'TIPO_TRIBUTARIO': str,
                             'DOC_TRIBUTARIO': str,
                             'TIPO_DOC': str,
                             'NUM_DOC': str,
                             'TIPO_PERSONA': str,
                             'RESIDENCIA': str,
                             'MAGNITUD': str,
                             'ACCIONISTA': str,
                             'RELACION': str,
                             'PAIS': str,
                             'GENERO': str,
                             'EDOCIVIL': str,
                             'ESPACIO': str,
                             'APE_PAT': str,
                             'APE_MAT': str,
                             'PRIMER_NOMBRE': str,
                             'SEGUNDO_NOMBRE': str})
    
    #Insertando datos
    for df in df_full:    
      df = df.fillna(' ')
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      values = df.values.tolist()
      self.cursor.fast_executemany = True
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()   
    
    
  def cargarInterfaceTcFormatoRcdV(self):
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [12,80,11,1,12,10,2,2,1,10,4,15,1,1,1,1,2,1,4,12,1,1,3,1,8,2,12]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['NU_CUENTA','NOMBRE','RUC','CO_TIPO_DOC','NU_DOC','CO_SBS','CO_ZONA_REG','CO_OFI_REG','CO_TIPO_IDENTIFICACION','NU_PARTIDA_FICHA','CO_CIUU','SIGLAS','RESIDENCIA','MAGNITUD_EMP','ACCIONISTA','REL_LAB_EMP','CLAS_DIAS','SITU_CUENTA','PAIS_ORIGEN','RUT','EST_CIVIL','SEXO','ID_CARTERA','CO_TIPO_CLIENTE','FE_NAC','CO_TIP_DOC_IDENT_COMP','NU_DOC_IDENT_COMP'],
                          dtype = {'NU_CUENTA': str,
                                   'NOMBRE': str,
                                   'RUC': str,
                                   'CO_TIPO_DOC': str,
                                   'NU_DOC': str,
                                   'CO_SBS': str,
                                   'CO_ZONA_REG': str,
                                   'CO_OFI_REG': str,
                                   'CO_TIPO_IDENTIFICACION': str,
                                   'NU_PARTIDA_FICHA': str,
                                   'CO_CIUU': str,
                                   'SIGLAS': str,
                                   'RESIDENCIA': str,
                                   'MAGNITUD_EMP': str,
                                   'ACCIONISTA': str,
                                   'REL_LAB_EMP': str,
                                   'CLAS_DIAS': str,
                                   'SITU_CUENTA': str,
                                   'PAIS_ORIGEN': str,
                                   'RUT': str,
                                   'EST_CIVIL': str,
                                   'SEXO': str,
                                   'ID_CARTERA': str,
                                   'CO_TIPO_CLIENTE': str,
                                   'FE_NAC': str,
                                   'CO_TIP_DOC_IDENT_COMP': str,
                                   'NU_DOC_IDENT_COMP': str})
    
    #Insertando datos        
    for df in df_full:    
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close() 
    
    
  def cargarInterfaceTcSaldosRcdVOld(self):  

    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [12,3,9,2,15,4,1,3,8,3,12,4,6,2,2,2,2,14]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['NRO_CUENTA_CMR','CODIGO_PRODUCTO','CODIGO_CLIENTE','TIPO_CREDITO','SALDO_NORMAL','DIAS_MORA','NEGATIVO','NUM_CUOTA','FECHA_VENC','ID_CARTERA','NRO_OPERACION','COD_OFI_EMP','UBIGEO_OFI_EMP','FLAG_TIP_IMPO','FLAG_ESTA_CTA','FLAG_TIP_PROD','FLAG_USO_PROD','COD_CTA_CONTABLE'],
                          dtype = {'NRO_CUENTA_CMR': str,
                                   'CODIGO_PRODUCTO': str,
                                   'CODIGO_CLIENTE': str,
                                   'TIPO_CREDITO': str,
                                   'SALDO_NORMAL': str,
                                   'DIAS_MORA': str,
                                   'NEGATIVO': str,
                                   'NUM_CUOTA': str,
                                   'FECHA_VENC': str,
                                   'ID_CARTERA': str,
                                   'NRO_OPERACION': str,
                                   'COD_OFI_EMP': str,
                                   'UBIGEO_OFI_EMP': str,
                                   'FLAG_TIP_IMPO': str,
                                   'FLAG_ESTA_CTA': str,
                                   'FLAG_TIP_PROD': str,
                                   'FLAG_USO_PROD': str,
                                   'COD_CTA_CONTABLE': str})  
    
    #Insertando datos
    for df in df_full:
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()  
    
    self.cursor.close() 
    
  def cargarInterfaceTcSaldosRcdV(self):       

    spark = SparkSession.builder.appName("Cargar interface TcSaldosRcdV").getOrCreate()
    df = spark.read.text('/mnt/ereportinterfaz/inseir/' + self.archivo_entrada)
    df = df.select(
      df.value.substr(1,12).alias('NRO_CUENTA_CMR'),
      df.value.substr(13,3).alias('CODIGO_PRODUCTO'),
      df.value.substr(16,9).alias('CODIGO_CLIENTE'),
      df.value.substr(25,2).alias('TIPO_CREDITO'),
      df.value.substr(27,15).alias('SALDO_NORMAL'),
      df.value.substr(42,4).alias('DIAS_MORA'),
      df.value.substr(45,1).alias('NEGATIVO'),
      df.value.substr(48,3).alias('NUM_CUOTA'),
      df.value.substr(50,8).alias('FECHA_VENC'),
      df.value.substr(58,3).alias('ID_CARTERA'),
      df.value.substr(61,12).alias('NRO_OPERACION'),
      df.value.substr(73,4).alias('COD_OFI_EMP'),
      df.value.substr(77,6).alias('UBIGEO_OFI_EMP'),
      df.value.substr(83,2).alias('FLAG_TIP_IMPO'),
      df.value.substr(85,2).alias('FLAG_ESTA_CTA'),
      df.value.substr(87,2).alias('FLAG_TIP_PROD'),
      df.value.substr(89,2).alias('FLAG_USO_PROD'),
      df.value.substr(91,14).alias('COD_CTA_CONTABLE'))
    #df = df.withColumn('FECHA_ACTUALIZACION', lit(self.fecha_actualizacion))
       
    url = "jdbc:sqlserver://auqui.database.windows.net;database=dbnormativo"
    usuario = "administrador"
    clave = "Auqui$2020"
    try:
      df.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("truncate", "true").option("url", url).option("dbtable", self.tabla_destino).option("user", usuario).option("password", clave).option("tableLock", "true").save()
    except:
        raise;
    
    
  def cargarInterfacePifRcdNoCmrV(self):    
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [6,12,3,9,12,18,18,18,4,4,2,2,1,2,4,6]
    df_full = pd.read_fwf(file_name, 
                         widths = fwidths,
                         chunksize = 15000,
                         names = ['PERIODO','CUENTA_CMR','PRODUCTO_SBS','CODIGO_RUT','NRO_CREDITO','CAPITAL_CASTIGADO','INTERES_CASTIGADO','COMISION_CASTIGADO','NRO_DIAS_MORA','COND_DIAS','COND_ESPECIAL','COND_DISPONIBILIDAD','TIPO_CARTERA','ESTADO_CUENTA','COD_AGENCIA','COD_UBIGEO'],
                         dtype = {'PERIODO': str,
                                  'CUENTA_CMR': str,
                                  'PRODUCTO_SBS': str,
                                  'CODIGO_RUT': str,
                                  'NRO_CREDITO': str,
                                  'CAPITAL_CASTIGADO': str,
                                  'INTERES_CASTIGADO': str,
                                  'COMISION_CASTIGADO': str,
                                  'NRO_DIAS_MORA': str,
                                  'COND_DIAS': str,
                                  'COND_ESPECIAL': str,
                                  'COND_DISPONIBILIDAD': str,
                                  'TIPO_CARTERA': str,
                                  'ESTADO_CUENTA': str,
                                  'COD_AGENCIA': str,
                                  'COD_UBIGEO': str})
        
    #Insertando datos
    for df in df_full:          
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close() 
    
    
  def cargarInterfaceIssacRcdNoCmr(self):    
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [6,12,3,9,12,18,18,18,4,4,2,2,1,2,4,6]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['PERIODO','CUENTA_CMR','PRODUCTO_SBS','CODIGO_RUT','NRO_CREDITO','CAPITAL_CASTIGADO','INTERES_CASTIGADO','COMISION_CASTIGADO','NRO_DIAS_MORA','COND_DIAS','COND_ESPECIAL','COND_DISPONIBILIDAD','TIPO_CARTERA','ESTADO_CUENTA','COD_AGENCIA','COD_UBIGEO'],
                          dtype = {'PERIODO': str,
                                   'CUENTA_CMR': str,
                                   'PRODUCTO_SBS': str,
                                   'CODIGO_RUT': str,
                                   'NRO_CREDITO': str,
                                   'CAPITAL_CASTIGADO': str,
                                   'INTERES_CASTIGADO': str,
                                   'COMISION_CASTIGADO': str,
                                   'NRO_DIAS_MORA': str,
                                   'COND_DIAS': str,
                                   'COND_ESPECIAL': str,
                                   'COND_DISPONIBILIDAD': str,
                                   'TIPO_CARTERA': str,
                                   'ESTADO_CUENTA': str,
                                   'COD_AGENCIA': str,
                                   'COD_UBIGEO': str})
   
    #Insertando datos
    for df in df_full: 
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
    
    
  def cargarInterfaceParientes(self):    
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [6,1,12,40,40,40,2,1,15]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['CORRELATIVO','TIPO_DOCUMENTO','NUMERO_DOCUMENTO','APELLIDO_PATERNO','APELLIDO_MATERNO','NOMBRES','PARENTESCO','CAMPO_1','CAMPO_2'],
                          dtype = {'CORRELATIVO': str,
                                    'TIPO_DOCUMENTO': str,
                                    'NUMERO_DOCUMENTO': str,
                                    'APELLIDO_PATERNO': str,
                                    'APELLIDO_MATERNO': str,
                                    'NOMBRES': str,
                                    'PARENTESCO': str,
                                    'CAMPO_1': str,
                                    'CAMPO_2': str})
    
    #Insertando datos
    for df in df_full: 
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
    
    
  def cargarInterfaceCastSalVencidosOld(self):
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [8,12,3,9,1,12,120,20,15,4,22,15,8]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['FECHA','CTA_CMR','COD_PRODUCTO','COD_CLIENTE','TIPO_DOC','NUM_DOC','NOMBRE','COD_OPERACION','SALDO','DIAS_MORA','IMPORTE','DEUDA_TOTAL','FEC_CASTIGO'],
                          dtype = {'FECHA': str,
                                   'CTA_CMR': str,
                                   'COD_PRODUCTO': str,
                                   'COD_CLIENTE': str,
                                   'TIPO_DOC': str,
                                   'NUM_DOC': str,
                                   'NOMBRE': str,
                                   'COD_OPERACION': str,
                                   'SALDO': str,
                                   'DIAS_MORA': str,
                                   'IMPORTE': str,
                                   'DEUDA_TOTAL': str,
                                   'FEC_CASTIGO': str})    
    
    #Insertando datos
    for df in df_full: 
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
    
  def cargarInterfaceCastSalVencidos(self):

      spark = SparkSession.builder.appName("Cargar interface CAST_SAL_VENCIDOS").getOrCreate()
      df = spark.read.text('/mnt/ereportinterfaz/inseir/' + self.archivo_entrada)
      df = df.select(
        df.value.substr(1,8).alias('FECHA'),    
        df.value.substr(9,12).alias('CTA_CMR'),    
        df.value.substr(21,3).alias('COD_PRODUCTO'),    
        df.value.substr(24,9).alias('COD_CLIENTE'),  
        df.value.substr(33,1).alias('TIPO_DOC'),  
        df.value.substr(34,12).alias('NUM_DOC'),  
        df.value.substr(46,120).alias('NOMBRE'),  
        df.value.substr(166,20).alias('COD_OPERACION'),  
        df.value.substr(186,15).alias('SALDO'),  
        df.value.substr(201,4).alias('DIAS_MORA'),  
        df.value.substr(204,22).alias('IMPORTE'),  
        df.value.substr(227,15).alias('DEUDA_TOTAL'),  
        df.value.substr(242,8).alias('FEC_CASTIGO'))
      url = "jdbc:sqlserver://auqui.database.windows.net;database=dbnormativo"
      usuario = "administrador"
      clave = "Auqui$2020"
      try:
        df.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("truncate", "true").option("url", url).option("dbtable", self.tabla_destino).option("user", usuario).option("password", clave).option("tableLock", "true").save()
      except:
          raise;
    
    
  def cargarInterfaceRcdRefinaDDMM(self):    
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [12,14,3]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['CUENTA_CMR',
                                    'SALDO_DIFERIDO',
                                    'COD_SBS'],
                          dtype = {'CUENTA_CMR': str,
                                    'SALDO_DIFERIDO': str,
                                    'COD_SBS': str})    
    
    #Insertando datos
    for df in df_full:     
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?)"
      values = df.values.tolist()
      self.cursor.fast_executemany = True
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
    
    
  def cargarInterfaceRcdRefinaAAMM(self):
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [12,14,3,12]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['CUENTA_CMR','INT_DIFERIDO','COD_SBS','NUMERO_OPERACION'],
                          dtype = {'CUENTA_CMR': str,
                                   'INT_DIFERIDO': str,
                                   'COD_SBS': str,
                                   'NUMERO_OPERACION': str})
    
    #Insertando datos        
    for df in df_full:     
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
    
    
  def cargarInterfacePifOcredigen(self):
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name, header = None, 
                          index_col=False, 
                          sep = '|', 
                          chunksize = 15000, 
                          names = ['CREDITO','NUMERO_CUENTA','COD_DEUD_RCD','NUM_CTA','PRODUCTO','MONEDA','MTO_CRE','SITUACION_OPERACION','MODALIDAD_PAGO','FECHA_VENCIMIENTO','CLASIFICACION_SBIF','TIPO_CREDITO','FECHA_ACTIVACION','MTO_DES','RENOVACION','CUOTAS','FECHA_ULT_RENEGOCIACION','MONTO_ULT_RENEGOCIACION','PLAZO_ULT_RENEGOCIACION','SALDO_CUOTAS','INTE_DIF','TOTAL_INTERES_DEVENGADO','MONTO_INTERES_JUDICIAL','FECHA_DDMMYYYY','PERIODICIDAD','RENOVACIONES','NUM_OPERACIONES','AGENCIA','TASA','DIAS_GRACIA','FECHA_PAGO_ULT','CUOTAS_PAGADAS'], 
                          dtype = {'CREDITO': str,
                                   'NUMERO_CUENTA': str,
                                   'COD_DEUD_RCD': str,
                                   'NUM_CTA': str,
                                   'PRODUCTO': str,
                                   'MONEDA': str,
                                   'MTO_CRE': str,
                                   'SITUACION_OPERACION': str,
                                   'MODALIDAD_PAGO': str,
                                   'FECHA_VENCIMIENTO': str,
                                   'CLASIFICACION_SBIF': str,
                                   'TIPO_CREDITO': str,
                                   'FECHA_ACTIVACION': str,
                                   'MTO_DES': str,
                                   'RENOVACION': str,
                                   'CUOTAS': str,
                                   'FECHA_ULT_RENEGOCIACION': str,
                                   'MONTO_ULT_RENEGOCIACION': str,
                                   'PLAZO_ULT_RENEGOCIACION': str,
                                   'SALDO_CUOTAS': str,
                                   'INTE_DIF': str,
                                   'TOTAL_INTERES_DEVENGADO': str,
                                   'MONTO_INTERES_JUDICIAL': str,
                                   'FECHA_DDMMYYYY': str,
                                   'PERIODICIDAD': str,
                                   'RENOVACIONES': str,
                                   'NUM_OPERACIONES': str,
                                   'AGENCIA': str,
                                   'TASA': str,
                                   'DIAS_GRACIA': str,
                                   'FECHA_PAGO_ULT': str,
                                   'CUOTAS_PAGADAS': str})      
    
    #Insertando datos
    for df in df_full:     
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
  
  #def cargarInterfaceRcc2(self):
    
    
  def cargarInterfaceRcc(self):
    
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name, header = None, names = ['LINEA'], chunksize = 15000, encoding = 'unicode_escape')
    
    ###### ------ Rcc ------ ######
    #Limpiando la tabla
    truncate_rcc = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate_rcc)
    self.connection.commit()
       
    #Generando datos
    for df in df_full:  
      df_rcc = df[(df['LINEA'].str[:1] == '1')]    
      df_rcc[['CODIGO_DEUDOR_SBS',
              'FECHA_PROCESO',
              'TIPO_DOCUMENTO_TRIBUTARIO',
              'DOCUMENTO_TRIBUTARIO',
              'TIPO_DOCUMENTO_IDENTIDAD',
              'DOCUMENTO_IDENTIDAD',
              'TIPO_PERSONA',
              'TIPO_EMPRESA',
              'CANTIDAD_EMPRESAS',
              'DEUDA_CALIFICACION_0',
              'DEUDA_CALIFICACION_1',
              'DEUDA_CALIFICACION_2',
              'DEUDA_CALIFICACION_3',
              'DEUDA_CALIFICACION_4',
              'APELLIDO_PATERNO_RAZON_SOCIAL',
              'APELLIDO_MATERNO',
              'APELLIDO_CASADA',
              'PRIMER_NOMBRE',
              'SEGUNDO_NOMBRE']] = df_rcc['LINEA'].str.split('|', expand=True) 
      del df_rcc['LINEA']
      df_rcc.loc[:, 'FECHA_PROCESO']='20201231'##eliminar para procesar archivo sin enmascarar
      df_rcc.loc[:, 'CODIGO_DEUDOR_SBS'] = df_rcc['CODIGO_DEUDOR_SBS'].str[2:]
      df_rcc.loc[:, 'TIPO_EMPRESA'] = df_rcc['TIPO_EMPRESA'].str.strip().str[:1]
      df_rcc.loc[:, 'FECHA_PROCESO']=pd.to_datetime(df_rcc['FECHA_PROCESO'], format='%Y%m%d').dt.date
      df_rcc.loc[:, 'FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df_rcc = df_rcc.where(pd.notnull(df_rcc), None)
      df_rcc = df_rcc.fillna(0)
      df_rcc['CODIGO_DEUDOR_SBS'] = pd.to_numeric(df_rcc['CODIGO_DEUDOR_SBS'], errors='coerce').replace(np.nan,0)
      df_rcc['CANTIDAD_EMPRESAS'] = pd.to_numeric(df_rcc['CANTIDAD_EMPRESAS'], errors='coerce').replace(np.nan,0)
      df_rcc['DEUDA_CALIFICACION_0'] = pd.to_numeric(df_rcc['DEUDA_CALIFICACION_0'], errors='coerce').replace(np.nan,0)
      df_rcc['DEUDA_CALIFICACION_1'] = pd.to_numeric(df_rcc['DEUDA_CALIFICACION_1'], errors='coerce').replace(np.nan,0)
      df_rcc['DEUDA_CALIFICACION_2'] = pd.to_numeric(df_rcc['DEUDA_CALIFICACION_2'], errors='coerce').replace(np.nan,0)
      df_rcc['DEUDA_CALIFICACION_3'] = pd.to_numeric(df_rcc['DEUDA_CALIFICACION_3'], errors='coerce').replace(np.nan,0)
      df_rcc['DEUDA_CALIFICACION_4'] = pd.to_numeric(df_rcc['DEUDA_CALIFICACION_4'], errors='coerce').replace(np.nan,0)

      #Insertando datos
      insert_rcc = "INSERT INTO [dbo].[" + self.tabla_destino + "] (CODIGO_DEUDOR_SBS,FECHA_PROCESO,TIPO_DOCUMENTO_TRIBUTARIO,DOCUMENTO_TRIBUTARIO,TIPO_DOCUMENTO_IDENTIDAD,DOCUMENTO_IDENTIDAD,TIPO_PERSONA,TIPO_EMPRESA,CANTIDAD_EMPRESAS,DEUDA_CALIFICACION_0,DEUDA_CALIFICACION_1,DEUDA_CALIFICACION_2,DEUDA_CALIFICACION_3,DEUDA_CALIFICACION_4,APELLIDO_PATERNO_RAZON_SOCIAL,APELLIDO_MATERNO,APELLIDO_CASADA,PRIMER_NOMBRE,SEGUNDO_NOMBRE, FECHA_ACTUALIZACION) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      self.cursor.executemany(insert_rcc, df_rcc.values.tolist())
      self.connection.commit()        
    
    ###### ------ Rcc Saldos ------ ######       
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name, header = None, names = ['LINEA'], chunksize = 15000, encoding = 'unicode_escape')
    
    #Limpiando la tabla
    truncate_rcc_saldos = "TRUNCATE TABLE " + self.tabla_destino_2
    self.cursor.execute(truncate_rcc_saldos)
    self.connection.commit() 
    
    #Generando datos
    for df in df_full:  
      df_rcc_saldos = df[(df['LINEA'].str[:1] == '2')]
      df_rcc_saldos.loc[:, 'CODIGO_DEUDOR_SBS'] = (df_rcc_saldos['LINEA'].str[2:11]).astype(int)   
      #df_rcc_saldos.loc[:, 'FECHA_PROCESO'] = df_rcc.loc[0,'FECHA_PROCESO']  
      df_rcc_saldos.loc[:, 'FECHA_PROCESO']='20201231'##eliminar para procesar archivo sin enmascarar
      df_rcc_saldos.loc[:, 'FECHA_PROCESO']=pd.to_datetime(df_rcc_saldos['FECHA_PROCESO'], format='%Y%m%d').dt.date
      df_rcc_saldos.loc[:, 'EMPRESA'] = df_rcc_saldos['LINEA'].str[12:16]
      df_rcc_saldos.loc[:, 'TIPO_CREDITO_SBS'] = df_rcc_saldos['LINEA'].str[17:17]
      df_rcc_saldos.loc[:, 'CUENTA_CONTABLE'] = df_rcc_saldos['LINEA'].str[18:31]
      df_rcc_saldos.loc[:, 'DIAS_ATRASO'] = df_rcc_saldos['LINEA'].str[32:35]
      df_rcc_saldos.loc[:, 'IMPORTE'] = df_rcc_saldos['LINEA'].str[36:50]
      df_rcc_saldos.loc[:, 'CLASIFICACION_DEUDOR'] = df_rcc_saldos['LINEA'].str[51:51]
      df_rcc_saldos.loc[:, 'FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str    
      df_rcc_saldos = df_rcc_saldos.drop('LINEA', 1)

      #Join con Rcc para obtener el ID
      select_rcc_act = ("SELECT * FROM " + self.tabla_destino + " WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'")
      sql_rcc_act = pd.read_sql_query(select_rcc_act, self.connection)
      df_rcc_act = pd.DataFrame(sql_rcc_act)
      df_rcc_saldos_id = df_rcc_act.merge(df_rcc_saldos, on=['CODIGO_DEUDOR_SBS','FECHA_PROCESO'], how='inner')
      df_rcc_saldos.index = df_rcc_saldos_id.index #Homogeneizando índices para igualar columnas
      df_rcc_saldos['RCC'] = df_rcc_saldos_id['ID']    
      df_rcc_saldos = df_rcc_saldos.drop(['CODIGO_DEUDOR_SBS', 'FECHA_PROCESO'], 1)
      df_rcc_saldos = df_rcc_saldos.where(pd.notnull(df_rcc_saldos), None)
    
      #Insertando datos
      insert_rcc_saldos = "INSERT INTO [dbo].[" + self.tabla_destino_2 + "] (EMPRESA,TIPO_CREDITO_SBS,CUENTA_CONTABLE,DIAS_ATRASO,IMPORTE,CLASIFICACION_DEUDOR,FECHA_ACTUALIZACION,RCC) VALUES (?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      self.cursor.executemany(insert_rcc_saldos, df_rcc_saldos.values.tolist())
      self.connection.commit()  
      
    self.cursor.close()   
    
    
  def cargarInterfacePifOcredigenV(self):
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name,
                     header = None,
                     index_col=False,
                     sep = '|',
                     chunksize = 15000, 
                     names = ['OPERACION','COD_DEUD_EMP','COD_DEUD_RCD','NUM_CTA','COD_PRO','COD_MON','MTO_CRE','SIT_OPE','MOD_PAG','FEC_VEN','CLA_DEU','TIP_CRE','FEC_DES','MTO_DES','RENOPE','PLZCRO','FEC_U_RE','MTO_U_RE','PLZ_U_RE','RENDEV','INTE_DIF','INT_SUS','INT_JUD','FEC_GEN','TIPO_CLIENTE','PERIODICIDAD','RENOVACIONES','NUM_OPERA_DEUD','COD_AGENCIA','TASA_INTERES','DIAS_GRACIA','FECHA_PAGO_ULT','NUM_CUOTAS'],
                    dtype = {'OPERACION': str,
                             'COD_DEUD_EMP': str,
                             'COD_DEUD_RCD': str,
                             'NUM_CTA': str,
                             'COD_PRO': str,
                             'COD_MON': str,
                             'MTO_CRE': str,
                             'SIT_OPE': str,
                             'MOD_PAG': str,
                             'FEC_VEN': str,
                             'CLA_DEU': str,
                             'TIP_CRE': str,
                             'FEC_DES': str,
                             'MTO_DES': str,
                             'RENOPE': str,
                             'PLZCRO': str,
                             'FEC_U_RE': str,
                             'MTO_U_RE': str,
                             'PLZ_U_RE': str,
                             'RENDEV': str,
                             'INTE_DIF': str,
                             'INT_SUS': str,
                             'INT_JUD': str,
                             'FEC_GEN': str,
                             'TIPO_CLIENTE': str,
                             'PERIODICIDAD': str,
                             'RENOVACIONES': str,
                             'NUM_OPERA_DEUD': str,
                             'COD_AGENCIA': str,
                             'TASA_INTERES': str,
                             'DIAS_GRACIA': str,
                             'FECHA_PAGO_ULT': str,
                             'NUM_CUOTAS': str})     
    
    #Insertando datos
    for df in df_full:  
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()
    
    
  def cargarInterfaceOcredigen(self):    
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name, 
                          header = None, 
                          index_col=False, 
                          sep = '|', 
                          chunksize = 15000, 
                          names = ['OPERACION','COD_DEUD_EMP','COD_DEUD_RCD','NUM_CTA','COD_PRO','COD_MON','MTO_CRE','SIT_OPE','MOD_PAG','FEC_VEN','CLA_DEU','TIP_CRE','FEC_DES','MTO_DES','RENOPE','PLZCRO','FEC_U_RE','MTO_U_RE','PLZ_U_RE','RENDEV','INTE_DIF','INT_SUS','INT_JUD','FEC_GEN','TIPO_CLIENTE','PERIODICIDAD','RENOVACIONES','NUM_OPERA_DEUD','COD_AGENCIA','TASA_INTERES','DIAS_GRACIA','FECHA_PAGO_ULT','NUM_CUOTAS'],
                         dtype = {'OPERACION': str,
                                  'COD_DEUD_EMP': str,
                                  'COD_DEUD_RCD': str,
                                  'NUM_CTA': str,
                                  'COD_PRO': str,
                                  'COD_MON': str,
                                  'MTO_CRE': str,
                                  'SIT_OPE': str,
                                  'MOD_PAG': str,
                                  'FEC_VEN': str,
                                  'CLA_DEU': str,
                                  'TIP_CRE': str,
                                  'FEC_DES': str,
                                  'MTO_DES': str,
                                  'RENOPE': str,
                                  'PLZCRO': str,
                                  'FEC_U_RE': str,
                                  'MTO_U_RE': str,
                                  'PLZ_U_RE': str,
                                  'RENDEV': str,
                                  'INTE_DIF': str,
                                  'INT_SUS': str,
                                  'INT_JUD': str,
                                  'FEC_GEN': str,
                                  'TIPO_CLIENTE': str,
                                  'PERIODICIDAD': str,
                                  'RENOVACIONES': str,
                                  'NUM_OPERA_DEUD': str,
                                  'COD_AGENCIA': str,
                                  'TASA_INTERES': str,
                                  'DIAS_GRACIA': str,
                                  'FECHA_PAGO_ULT': str,
                                  'NUM_CUOTAS': str})

    #Insertando datos
    for df in df_full:     
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close()    
    
    
  def cargarInterfaceIssacOcredigenVOld(self):
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name,
                          header = None,
                          index_col=False,
                          sep = '|',
                          chunksize = 15000, 
                          names = ['OPERACION','COD_DEUDEMP','COD_DEUDRCD','NUM_CTA','COD_PRO','COD_MON','MTO_CRE','SIT_OPE','MOD_PAG','FEC_VEN','CLA_DEU','TIP_CRE','FEC_DES','MTO_DES','RENOPE','PLZCRO','FEC_U_RE','MTO_U_RE','PLZ_U_RE','RENDEV','INTE_DIF','INT_SUS','INT_JUD','FEC_GEN','TIPO_CLIENTE','PERIODICIDAD','RENOVACIONES','NUM_OPERA_DEUD','COD_AGENCIA','TASA_INTERES','DIAS_GRACIA','FECHA_PAGO_ULT','NUM_CUOTAS'], 
                          dtype = {'OPERACION': str,
                                    'COD_DEUDEMP': str,
                                    'COD_DEUDRCD': str,
                                    'NUM_CTA': str,
                                    'COD_PRO': str,
                                    'COD_MON': str,
                                    'MTO_CRE': str,
                                    'SIT_OPE': str,
                                    'MOD_PAG': str,
                                    'FEC_VEN': str,
                                    'CLA_DEU': str,
                                    'TIP_CRE': str,
                                    'FEC_DES': str,
                                    'MTO_DES': str,
                                    'RENOPE': str,
                                    'PLZCRO': str,
                                    'FEC_U_RE': str,
                                    'MTO_U_RE': str,
                                    'PLZ_U_RE': str,
                                    'RENDEV': str,
                                    'INTE_DIF': str,
                                    'INT_SUS': str,
                                    'INT_JUD': str,
                                    'FEC_GEN': str,
                                    'TIPO_CLIENTE': str,
                                    'PERIODICIDAD': str,
                                    'RENOVACIONES': str,
                                    'NUM_OPERA_DEUD': str,
                                    'COD_AGENCIA': str,
                                    'TASA_INTERES': str,
                                    'DIAS_GRACIA': str,
                                    'FECHA_PAGO_ULT': str,
                                    'NUM_CUOTAS': str})
    
    #Insertando datos
    for df in df_full:        
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit()    
    self.cursor.close() 
    
  def cargarInterfaceIssacOcredigenV(self): 
          
    schema = StructType([
    StructField("OPERACION", StringType(), True),
    StructField("COD_DEUDEMP", StringType(), True),
    StructField("COD_DEUDRCD", StringType(), True),
    StructField("NUM_CTA", StringType(), True),
    StructField("COD_PRO", StringType(), True),
    StructField("COD_MON", StringType(), True),
    StructField("MTO_CRE", StringType(), True),
    StructField("SIT_OPE", StringType(), True),
    StructField("MOD_PAG", StringType(), True),
    StructField("FEC_VEN", StringType(), True),
    StructField("CLA_DEU", StringType(), True),
    StructField("TIP_CRE", StringType(), True),
    StructField("FEC_DES", StringType(), True),
    StructField("MTO_DES", StringType(), True),
    StructField("RENOPE", StringType(), True),
    StructField("PLZCRO", StringType(), True),
    StructField("FEC_U_RE", StringType(), True),
    StructField("MTO_U_RE", StringType(), True),
    StructField("PLZ_U_RE", StringType(), True),
    StructField("RENDEV", StringType(), True),
    StructField("INTE_DIF", StringType(), True),
    StructField("INT_SUS", StringType(), True),
    StructField("INT_JUD", StringType(), True),
    StructField("FEC_GEN", StringType(), True),
    StructField("TIPO_CLIENTE", StringType(), True),
    StructField("PERIODICIDAD", StringType(), True),
    StructField("RENOVACIONES", StringType(), True),
    StructField("NUM_OPERA_DEUD", StringType(), True),
    StructField("COD_AGENCIA", StringType(), True),
    StructField("TASA_INTERES", StringType(), True),
    StructField("DIAS_GRACIA", StringType(), True),
    StructField("NUM_CUOTAS", StringType(), True),
    StructField("FECHA_PAGO_ULT", StringType(), True)])

    spark = SparkSession.builder.appName("Cargar interface IssacOcredigenV").getOrCreate()
    df = spark.read.option("delimiter", "|").csv('/mnt/ereportinterfaz/inseir/' + self.archivo_entrada, header=False,schema=schema)
    display(df)
       
    url = "jdbc:sqlserver://auqui.database.windows.net;database=dbnormativo"
    usuario = "administrador"
    clave = "Auqui$2020"
    try:
      df.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("truncate", "true").option("url", url).option("dbtable", self.tabla_destino).option("user", usuario).option("password", clave).option("tableLock", "true").save()
    except:
        raise;
    
    
  def cargarInterfaceOcredicuo(self):
        
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface    
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name,
                          header = None,
                          index_col=False,
                          sep = '|',
                          chunksize = 15000, 
                          names = ['OPERACION','COD_DEU_EMP','COD_DEU_RCD','NUM_CTA','NUM_CUO','MTO_CAP','MTO_INT','MTO_COM','MTO_MOR','FEC_VEN','FEC_GEN','MONEDA'],
                          dtype = {'OPERACION': str,
                                   'COD_DEU_EMP': str,
                                   'COD_DEU_RCD': str,
                                   'NUM_CTA': str,
                                   'NUM_CUO': str,
                                   'MTO_CAP': str,
                                   'MTO_INT': str,
                                   'MTO_COM': str,
                                   'MTO_MOR': str,
                                   'FEC_VEN': str,
                                   'FEC_GEN': str,
                                   'MONEDA': str})
    
    #Insertando datos
    for df in df_full:    
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, df.values.tolist())
        self.connection.commit()    
    self.cursor.close() 

    
  def cargarInterfaceSaldosRcd(self):   
    
    #Limpiando la tabla
    truncate = "TRUNCATE TABLE " + self.tabla_destino
    self.cursor.execute(truncate)
    self.connection.commit()
    
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    fwidths = [6,12,3,9,12,18,18,18,4,4,2,2,1,2,18,3,4,6,1,18,18,16,1]
    df_full = pd.read_fwf(file_name, 
                          widths = fwidths,
                          chunksize = 15000,
                          names = ['PERIODO','CUENTA_CMR','COD_PRODUCTO','COD_CLIENTE','COD_OPERACION','SALDO_NORMAL','SALDO_VENCIDO','INTERES','DIAS_MORA','CONDICION_DIAS','CONDICION_ESPECIAL','CONDICION_DISPONIBILIDAD','TIPO_CARTERA','ESTADO_CUENTA','INTERES_VENCIDO','COD_SUCURSAL','COD_UBIGEO','ST_ESTADO_HIPOTECA','SALDO_PAGO_ADELANTADO','COMISION','LINEA_OTORGADA','LINEA_NO_UTILIZADA','SITUACION'],
                          dtype = {'PERIODO': str,
                                   'CUENTA_CMR': str,
                                   'COD_PRODUCTO': str,
                                   'COD_CLIENTE': str,
                                   'COD_OPERACION': str,
                                   'SALDO_NORMAL': str,
                                   'SALDO_VENCIDO': str,
                                   'INTERES': str,
                                   'DIAS_MORA': str,
                                   'CONDICION_DIAS': str,
                                   'CONDICION_ESPECIAL': str,
                                   'CONDICION_DISPONIBILIDAD': str,
                                   'TIPO_CARTERA': str,
                                   'ESTADO_CUENTA': str,
                                   'INTERES_VENCIDO': str,
                                   'COD_SUCURSAL': str,
                                   'COD_UBIGEO': str,
                                   'ST_ESTADO_HIPOTECA': str,
                                   'SALDO_PAGO_ADELANTADO': str,
                                   'COMISION': str,
                                   'LINEA_OTORGADA': str,
                                   'LINEA_NO_UTILIZADA': str,
                                   'SITUACION': str})
        
    #Insertando datos
    for df in df_full:  
      df['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
      df['SALDO_NORMAL'] = df['SALDO_NORMAL'].astype(float)
      df['SALDO_VENCIDO'] = df['SALDO_VENCIDO'].astype(float)
      df['INTERES'] = df['INTERES'].astype(float)
      df['INTERES_VENCIDO'] = df['INTERES_VENCIDO'].astype(float)
      df['SALDO_PAGO_ADELANTADO'] = df['SALDO_PAGO_ADELANTADO'].astype(float)
      df['COMISION'] = df['COMISION'].astype(float)
      df['LINEA_OTORGADA'] = df['LINEA_OTORGADA'].astype(float)
      df['LINEA_NO_UTILIZADA'] = df['LINEA_NO_UTILIZADA'].astype(float)

      df = df.where(pd.notnull(df), None)
      insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df.values.tolist()
      if values:
        self.cursor.executemany(insert, values)
        self.connection.commit() 
      
    self.cursor.close()      
    
    
  def actualizarCodigoDeudorSbs(self):   
        
    #Leyendo la interface
    file_name = self.ruta_origen + self.archivo_entrada
    df_full = pd.read_csv(file_name, header = None, names = ['LINEA'], chunksize = 15000)

    #Obteniendo datos
    chunks = []
    for df in df_full:  
      df['CODIGO_DEUDOR_SBS'] = df['LINEA'].str[11:20]
      df['TIPO_DOCUMENTO_IDENTIDAD'] = df['LINEA'].str[72:72]
      df['DOCUMENTO_IDENTIDAD'] = df['LINEA'].str[73:84]
      df = df.where(pd.notnull(df), None)
      df = df.drop(columns=['LINEA'])
      chunks.append(df)
    
    df_rcc_ok = pd.concat(chunks, ignore_index=True)
      
    #Obteniendo Código deudor SBS
    select_cliente = """SELECT TIPO_DOCUMENTO_IDENTIDAD, DOCUMENTO_IDENTIDAD FROM CLIENTE_DEUDOR"""
    sql_cliente = pd.read_sql_query(select_cliente, self.connection)
    df_cliente = pd.DataFrame(sql_cliente)    
    df_codigo_deudor_sbs = pd.merge(df_rcc_ok, df_cliente, on=['TIPO_DOCUMENTO_IDENTIDAD', 'DOCUMENTO_IDENTIDAD'], how='inner')
    
    #Actualizando Código deudor SBS
    update_codigo_deudor_sbs = "UPDATE [DBO].CLIENTE_DEUDOR SET CODIGO_DEUDOR_SBS = ? WHERE TIPO_DOCUMENTO_IDENTIDAD = ? AND DOCUMENTO_IDENTIDAD = ?"
    values = df_codigo_deudor_sbs.values.tolist()
    self.cursor.fast_executemany = True
    if values:
      self.cursor.executemany(update_codigo_deudor_sbs, values)
      self.connection.commit()    

  def cargarInterfaceRep14Hipotecario(self):
    try:
        file_name = self.ruta_origen + self.archivo_entrada
        df_full = pd.read_csv(file_name, header = None, index_col=False, sep = '|', chunksize = 15000, names = ['CUENTA',
                                                                                                                'NOMBRES',
                                                                                                                'TIPO_CREDITO',
                                                                                                                'ESTADO',
                                                                                                                'DIAS_ATRASO',
                                                                                                                'SITUACION',
                                                                                                                'COD_INSTITUCION',
                                                                                                                'CAPITAL_TOTAL',
                                                                                                                'CAPITA_NO_AMORTIZADO',
                                                                                                                'CAPITA_AMORTIZADO',
                                                                                                                'TIPO_CAMBIO',
                                                                                                                'TIPO_CLIENTE'],
                                                                                                        dtype = {'CUENTA':str ,
                                                                                                                 'NOMBRES':str,
                                                                                                                 'TIPO_CREDITO':str,
                                                                                                                 'ESTADO':str,
                                                                                                                 'DIAS_ATRASO':str,
                                                                                                                 'SITUACION':str,
                                                                                                                 'COD_INSTITUCION':str,
                                                                                                                 'CAPITAL_TOTAL':float,
                                                                                                                 'CAPITA_NO_AMORTIZADO':float,
                                                                                                                 'CAPITA_AMORTIZADO':float,
                                                                                                                 'TIPO_CAMBIO':float,
                                                                                                                 'TIPO_CLIENTE':str})
        
        
        #Limpiando la tabla
        truncate = "TRUNCATE TABLE " + self.tabla_destino
        self.cursor.execute(truncate)
        #self.connection.commit()
        
        for df in df_full:
            insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"
            self.cursor.fast_executemany = True
            self.cursor.executemany(insert, df.values.tolist())
            #self.connection.commit()  
        self.connection.commit()  
    except:
        self.connection.rollback() 
        raise
    finally:
        self.cursor.close()

  def cargarInterfaceRep14PifVencidos(self):
    try:
        file_name = self.ruta_origen + self.archivo_entrada
        df_full = pd.read_csv(file_name, header = None, index_col=False, sep = '|', chunksize = 15000, names = ['CUENTA',
                                                                                                                'NOMBRES',
                                                                                                                'TIPO_CREDITO',
                                                                                                                'ESTADO',
                                                                                                                'DIAS_ATRASO',
                                                                                                                'SITUACION',
                                                                                                                'COD_INSTITUCION',
                                                                                                                'CAPITAL_TOTAL',
                                                                                                                'CAPITA_NO_AMORTIZADO',
                                                                                                                'CAPITA_AMORTIZADO',
                                                                                                                'TIPO_CAMBIO',
                                                                                                                'TIPO_CLIENTE'],
                                                                                                        dtype = {'CUENTA':str ,
                                                                                                                 'NOMBRES':str,
                                                                                                                 'TIPO_CREDITO':str,
                                                                                                                 'ESTADO':str,
                                                                                                                 'DIAS_ATRASO':str,
                                                                                                                 'SITUACION':str,
                                                                                                                 'COD_INSTITUCION':str,
                                                                                                                 'CAPITAL_TOTAL':float,
                                                                                                                 'CAPITA_NO_AMORTIZADO':float,
                                                                                                                 'CAPITA_AMORTIZADO':float,
                                                                                                                 'TIPO_CAMBIO':float,
                                                                                                                 'TIPO_CLIENTE':str})
        
        
        #Limpiando la tabla
        truncate = "TRUNCATE TABLE " + self.tabla_destino
        self.cursor.execute(truncate)
        #self.connection.commit()
        
        for df in df_full:
            insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"
            self.cursor.fast_executemany = True
            self.cursor.executemany(insert, df.values.tolist())
            #self.connection.commit()  
        self.connection.commit()
    except:
        self.connection.rollback() 
        raise
    finally:
        self.cursor.close()
        
  def cargarInterfacePifRep14Hipotecario(self):
    try:
        file_name = self.ruta_origen + self.archivo_entrada
        df_full = pd.read_csv(file_name, header = None, index_col=False, sep = '|', chunksize = 15000, names = ['CUENTA_CMR',
                                                                                                                'NOMBRES',
                                                                                                                'CODIGO_SBS',
                                                                                                                'ESTADO',
                                                                                                                'DIAS_MORA',
                                                                                                                'SITUACION',
                                                                                                                'COD_INSTITUCION',
                                                                                                                'CAPITAL_TOTAL',
                                                                                                                'CAPITA_NO_AMORTIZADO',
                                                                                                                'CAPITA_AMORTIZADO',
                                                                                                                'TIPO_CAMBIO',
                                                                                                                'TIPO_CLIENTE'],
                                                                                                        dtype = {'CUENTA_CMR':str ,
                                                                                                                 'NOMBRES':str,
                                                                                                                 'CODIGO_SBS':str,
                                                                                                                 'ESTADO':str,
                                                                                                                 'DIAS_MORA':str,
                                                                                                                 'SITUACION':str,
                                                                                                                 'COD_INSTITUCION':str,
                                                                                                                 'CAPITAL_TOTAL':float,
                                                                                                                 'CAPITA_NO_AMORTIZADO':float,
                                                                                                                 'CAPITA_AMORTIZADO':float,
                                                                                                                 'TIPO_CAMBIO':float,
                                                                                                                 'TIPO_CLIENTE':str})
        
        
        #Limpiando la tabla
        truncate = "TRUNCATE TABLE " + self.tabla_destino
        self.cursor.execute(truncate)
        #self.connection.commit()
        
        for df in df_full:
            insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"
            self.cursor.fast_executemany = True
            self.cursor.executemany(insert, df.values.tolist())
            #self.connection.commit()  
        self.connection.commit()
    except:
        self.connection.rollback() 
        raise
    finally:
        self.cursor.close() 
        

  def cargarInterfaceRep14IssacVencidos(self):
    try:
        file_name = self.ruta_origen + self.archivo_entrada
        df_full = pd.read_csv(file_name, header = None, index_col=False, sep = '|', chunksize = 15000, names = ['CUENTA',
                                                                                                                'NOMBRES',
                                                                                                                'TIPO_CREDITO',
                                                                                                                'ESTADO',
                                                                                                                'DIAS_ATRASO',
                                                                                                                'SITUACION',
                                                                                                                'COD_INSTITUCION',
                                                                                                                'CAPITAL_TOTAL',
                                                                                                                'CAPITA_NO_AMORTIZADO',
                                                                                                                'CAPITA_AMORTIZADO',
                                                                                                                'TIPO_CAMBIO',
                                                                                                                'TIPO_CLIENTE'],
                                                                                                        dtype = {'CUENTA':str ,
                                                                                                                 'NOMBRES':str,
                                                                                                                 'TIPO_CREDITO':str,
                                                                                                                 'ESTADO':str,
                                                                                                                 'DIAS_ATRASO':str,
                                                                                                                 'SITUACION':str,
                                                                                                                 'COD_INSTITUCION':str,
                                                                                                                 'CAPITAL_TOTAL':float,
                                                                                                                 'CAPITA_NO_AMORTIZADO':float,
                                                                                                                 'CAPITA_AMORTIZADO':float,
                                                                                                                 'TIPO_CAMBIO':float,
                                                                                                                 'TIPO_CLIENTE':str})
        
        
        #Limpiando la tabla
        truncate = "TRUNCATE TABLE " + self.tabla_destino
        self.cursor.execute(truncate)
        #self.connection.commit()
        
        for df in df_full:
            insert = "INSERT INTO [dbo].[" + self.tabla_destino + "] VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"
            self.cursor.fast_executemany = True
            self.cursor.executemany(insert, df.values.tolist())
            #self.connection.commit()  
        self.connection.commit()  
    except:
        self.connection.rollback() 
        raise
    finally:
        self.cursor.close()        
        
#Ejecución de prueba
#objeto = PrcCargaInterfaces(12)
#objeto.cargarInterfacePifRep14Hipotecario()
#objeto = PrcCargaInterfaces(28)
#objeto.cargarInterfaceRep14IssacVencidos()
#objeto = PrcCargaInterfaces(14)
#objeto.cargarInterfaceRep14Hipotecario()
#objeto.cargarInterfaceRep14PifVencidos()
#objeto = PrcCargaInterfaces(9)
#objeto.cargarInterfaceFormatoRcd()
#objeto.cargarInterfaceSatMaeFusion()
#objeto.cargarInterfaceSatIndiceMorosidad()
#objeto.cargarInterfaceMancomunoIssac()
#objeto.cargarInterfaceTcFormatoRcdV()
#objeto.cargarInterfaceTcSaldosRcdV()
#objeto.cargarInterfacePifRcdNoCmrV()
#objeto.cargarInterfaceIssacRcdNoCmr()
#objeto.cargarInterfaceParientes()
#objeto = PrcCargaInterfaces(20)
#objeto.cargarInterfaceCastSalVencidos()
#objeto.cargarInterfaceRcdRefinaDDMM()
#objeto.cargarInterfaceRcdRefinaAAMM()
#objeto.cargarInterfacePifOcredigen()
#objeto.cargarInterfaceRcc()
#objeto.cargarInterfacePifOcredigenV()
#objeto.cargarInterfaceOcredigen()
#objeto = PrcCargaInterfaces(27)
#objeto.cargarInterfaceIssacOcredigenV()
#objeto.cargarInterfaceOcredicuo()
#objeto.cargarInterfaceSaldosRcd()
#objeto.actualizarCodigoDeudorSbs()